In [2]:
import pandas as pd
from datetime import datetime


#### Reformat data into old Fitbit export style.


In [3]:
filepath = r'~/Downloads/takeout-20231226T235127Z-001'
df = pd.read_json(f'{filepath}/Takeout/Fitbit/Global Export Data/sleep-2023-11-21.json') # Read 1 month at a time.

In [4]:
unnested = pd.json_normalize(df['levels'])

In [5]:
df = pd.merge(
    left=df[['startTime', 'endTime', 'minutesAsleep', 'minutesAwake', 'timeInBed']], 
    right=unnested.iloc[:, 2:], 
    left_index=True, 
    right_index=True
)

In [6]:
df.rename(columns={
    'summary.wake.count':'Number of Awakenings',
    'summary.rem.minutes': 'Minutes REM Sleep',
    'summary.light.minutes':'Minutes Light Sleep',
    'summary.deep.minutes':'Minutes Deep Sleep',
}, inplace=True)

In [7]:
tmp_col = df['Number of Awakenings'].copy()
df.drop(columns=['Number of Awakenings'], inplace=True)
df.insert(4, 'Number of Awakenings', tmp_col)

In [8]:
df = df[[
    'startTime', 
    'endTime', 
    'minutesAsleep', 
    'minutesAwake', 
    'Number of Awakenings', 
    'timeInBed', 
    'Minutes REM Sleep',
    'Minutes Light Sleep',
    'Minutes Deep Sleep',
]]

In [11]:
df = df.astype({'startTime':'datetime64[ns]', 'endTime':'datetime64[ns]'})

In [12]:
def convert_to_fitbit_time(cols):
    """Converts millisecond UTC timestamp into native 12h time."""
    for col in cols:
        df[col] = df[col].apply(lambda x: x.strftime('%Y-%m-%d %I:%M%p'))
        df[col] = df[col].apply(lambda x: x.replace(' 0', ' '))

        
convert_to_fitbit_time(['startTime', 'endTime'])

In [23]:
df.head()

,startTime,endTime,minutesAsleep,minutesAwake,Number of Awakenings,timeInBed,Minutes REM Sleep,Minutes Light Sleep,Minutes Deep Sleep
0,2023-12-21 12:49AM,2023-12-21 10:31AM,502,80,49.0,582,91.0,345.0,66.0
1,2023-12-20 12:47AM,2023-12-20 9:52AM,477,67,34.0,544,111.0,279.0,87.0
2,2023-12-19 1:28AM,2023-12-19 10:19AM,463,67,31.0,530,108.0,268.0,87.0
3,2023-12-18 12:15AM,2023-12-18 10:01AM,512,74,42.0,586,81.0,341.0,90.0
4,2023-12-17 1:11AM,2023-12-17 11:19AM,553,54,43.0,607,122.0,346.0,85.0


In [10]:
savepath = r'~/OneDrive/Documents/531_data'
df.to_csv(f'{savepath}/fitbit_sleep_export.csv', index=False, header=False, na_rep='N/A')